In [1]:
#Import Libraries
import pandas as pd
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv('ps6_trainvalid.csv')

# Data Preprocessing

In [3]:
#Drop first row of null values
data = data.drop(index=0)

In [4]:
data.describe()

,temperature,humidity,pressure,wind_direction,wind_speed
count,45010.000000,44861.000000,44761.000000,45012.000000,45012.000000
mean,290.841547,62.823767,1015.915127,139.733804,1.220652
std,6.465152,21.786687,11.316175,105.545576,1.435687
min,266.503667,5.000000,886.000000,0.000000,0.000000
25%,286.370000,48.000000,1013.000000,29.000000,0.000000
50%,290.530000,66.000000,1016.000000,140.000000,1.000000
75%,295.080000,81.000000,1021.000000,230.000000,2.000000
max,315.470000,100.000000,1044.000000,360.000000,17.000000


In [5]:
#filling null values for temp, pressure, humidity
data['temperature'] = data.temperature.fillna(method='bfill')
data['humidity'] = data.humidity.fillna(method='bfill')
data['pressure'] = data.pressure.fillna(method='bfill')

In [6]:
#Joining wind direction and wind speed to wind vector
wd_rad = data.wind_direction*np.pi/180
data['Wx'] = data.wind_speed*np.cos(wd_rad)
data['Wy'] = data.wind_speed*np.sin(wd_rad)

data = data.drop(['wind_direction','wind_speed'],axis=1)

In [7]:
#Convert date time into hour timestamp
import datetime
data['datetime'] = pd.to_datetime(data['datetime'])
timestamp_h = data['datetime'].map(datetime.datetime.timestamp)/3600

In [8]:
#Adding sin and cos terms for periodicity
day = 24
year = (365.2425)*day

data['Day sin'] = np.sin(timestamp_h * (2 * np.pi / day))
data['Day cos'] = np.cos(timestamp_h * (2 * np.pi / day))
data['Year sin'] = np.sin(timestamp_h * (2 * np.pi / year))
data['Year cos'] = np.cos(timestamp_h * (2 * np.pi / year))

In [9]:
#dropping datetime and weather
data = data.drop(['weather','datetime'],axis=1)

In [10]:
data

,temperature,humidity,pressure,Wx,Wy,Day sin,Day cos,Year sin,Year cos
1,291.870000,88.0,1013.0,0.000000,0.000000,-8.660254e-01,5.000000e-01,-0.999924,0.012325
2,291.868186,88.0,1013.0,0.000000,0.000000,-7.071068e-01,7.071068e-01,-0.999915,0.013041
3,291.862844,88.0,1013.0,0.000000,0.000000,-5.000000e-01,8.660254e-01,-0.999905,0.013758
4,291.857503,88.0,1013.0,0.000000,0.000000,-2.588190e-01,9.659258e-01,-0.999895,0.014475
5,291.852162,88.0,1013.0,0.000000,0.000000,-6.304366e-12,1.000000e+00,-0.999885,0.015192
...,...,...,...,...,...,...,...,...,...
45008,295.440000,17.0,1017.0,0.965926,-0.258819,8.660254e-01,5.000000e-01,-0.654451,0.756104
45009,296.020000,16.0,1016.0,0.965926,-0.258819,9.659258e-01,2.588190e-01,-0.653909,0.756573
45010,296.510000,17.0,1015.0,0.965926,-0.258819,1.000000e+00,1.086883e-11,-0.653367,0.757042
45011,297.090000,17.0,1014.0,0.000000,-0.000000,9.659258e-01,-2.588190e-01,-0.652824,0.757510


In [11]:
#Splitting into train, validation and test
column_indices = {name: i for i, name in enumerate(data.columns)}

n = len(data)
train_df = data[0:int(n*0.7)]
val_df = data[int(n*0.7):int(n*0.9)]
test_df = data[int(n*0.9):]

num_features = data.shape[1]

In [12]:
#Normalizing all the data
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [13]:
train_df.describe()

,temperature,humidity,pressure,Wx,Wy,Day sin,Day cos,Year sin,Year cos
count,3.150800e+04,3.150800e+04,3.150800e+04,3.150800e+04,3.150800e+04,3.150800e+04,3.150800e+04,3.150800e+04,3.150800e+04
mean,-6.534767e-14,-3.831587e-16,1.239591e-14,3.984368e-15,1.114190e-14,1.849902e-17,-2.873514e-17,4.327079e-16,-1.239410e-15
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.657681e+00,-2.560109e+00,-1.518860e+01,-1.126771e+01,-1.203217e+01,-1.414442e+00,-1.414068e+00,-1.437382e+00,-1.554323e+00
25%,-6.866147e-01,-6.908878e-01,-5.952103e-01,-5.896149e-01,-2.156248e-01,-1.000200e+00,-9.998950e-01,-1.003597e+00,-9.630392e-01
50%,-5.083055e-02,1.297458e-01,-1.355759e-01,9.163101e-02,2.357689e-01,-1.332905e-04,6.089053e-06,2.593580e-02,1.121578e-01
75%,6.731280e-01,8.136072e-01,6.687842e-01,9.163101e-02,2.357689e-01,9.999339e-01,9.999072e-01,1.017105e+00,9.786113e-01
max,3.372504e+00,1.771013e+00,2.966956e+00,1.386943e+01,8.471816e+00,1.414175e+00,1.414080e+00,1.364679e+00,1.322268e+00


# Single Step Prediction

## Dataset creation

In [14]:
# Function to create windows and labels
def create_dataset_single(train_df,window_size):
    train_windows = []
    train_labels = []
    
#     train_df = train_df.to_numpy()

    for i in range(len(train_df)-window_size):
        train_windows.append(train_df.iloc[i:i+window_size])
        train_labels.append(train_df['temperature'].iloc[i+window_size])
    
    return np.array(train_windows),np.array(train_labels)

In [15]:
#train,val and test for windows 24,72 and 120
train_windows24, train_labels24 = create_dataset_single(train_df,24)
train_windows72, train_labels72 = create_dataset_single(train_df,72)
train_windows120, train_labels120 = create_dataset_single(train_df,120)


val_windows24,val_labels24 = create_dataset_single(val_df,24)
val_windows72,val_labels72 = create_dataset_single(val_df,72)
val_windows120,val_labels120 = create_dataset_single(val_df,120)


test_windows24,test_labels24 = create_dataset_single(test_df,24)
test_windows72,test_labels72 = create_dataset_single(test_df,72)
test_windows120,test_labels120 = create_dataset_single(test_df,120)

## Baseline (Last Time Step)

In [16]:
def baseline(data):
    return data[:,-1,0]


baseline_test24 = mean_absolute_error(baseline(test_windows24),test_labels24)
baseline_test72 = mean_absolute_error(baseline(test_windows72),test_labels72)
baseline_test120 = mean_absolute_error(baseline(test_windows120),test_labels120)

In [17]:
baseline_test24,baseline_test72,baseline_test120

(0.13169526830479683, 0.13196778638959025, 0.13142798156378666)

## Linear Model

In [18]:
linear = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1)
])

In [19]:
#Compile and Fit function for all three windows
def compile_and_fit(model):
    model.compile(loss=tf.losses.MeanSquaredError(),metrics=[tf.metrics.MeanAbsoluteError()])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2, mode='min')
    
    print('training model 1')
    history24 = model.fit(train_windows24,train_labels24,validation_data=(val_windows24,val_labels24),
                          callbacks=[early_stopping],epochs=20)
    print()
    print('training model 2')
    
    history72 = model.fit(train_windows72,train_labels72,validation_data=(val_windows72,val_labels72),
                          callbacks=[early_stopping],epochs=20)
    print()
    print('training model 3')
    
    history120 = model.fit(train_windows120,train_labels120,validation_data=(val_windows120,val_labels120),
                          callbacks=[early_stopping],epochs=20)
    
    return history24,history72,history120

In [20]:
linear_history24,linear_history72,linear_history120 = compile_and_fit(linear)

training model 1
Epoch 1/20
984/984 [==============================] - 8s 7ms/step - loss: 1.8924 - mean_absolute_error: 1.0651 - val_loss: 0.5637 - val_mean_absolute_error: 0.6037
Epoch 2/20
984/984 [==============================] - 7s 7ms/step - loss: 0.5164 - mean_absolute_error: 0.5846 - val_loss: 0.5299 - val_mean_absolute_error: 0.5919
Epoch 3/20
984/984 [==============================] - 7s 7ms/step - loss: 0.5029 - mean_absolute_error: 0.5776 - val_loss: 0.5219 - val_mean_absolute_error: 0.5873
Epoch 4/20
984/984 [==============================] - 7s 7ms/step - loss: 0.5028 - mean_absolute_error: 0.5789 - val_loss: 0.5203 - val_mean_absolute_error: 0.5866
Epoch 5/20
984/984 [==============================] - 7s 7ms/step - loss: 0.5071 - mean_absolute_error: 0.5817 - val_loss: 0.5270 - val_mean_absolute_error: 0.5903
Epoch 6/20
984/984 [==============================] - 7s 7ms/step - loss: 0.5071 - mean_absolute_error: 0.5808 - val_loss: 0.5237 - val_mean_absolute_error: 0.5877

In [21]:
linear_loss24,linear_test24 = linear.evaluate(test_windows24,test_labels24)
linear_loss72,linear_test72 = linear.evaluate(test_windows72,test_labels72)
linear_loss120,linear_test120 = linear.evaluate(test_windows120,test_labels120)

137/137 [==============================] - 1s 4ms/step - loss: 0.4751 - mean_absolute_error: 0.5419


## Dense Model

In [22]:
dense_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dense(units=1),
])

In [23]:
dense_history24,dense_history72,dense_history120 = compile_and_fit(dense_model)

training model 1
Epoch 1/20
984/984 [==============================] - 12s 11ms/step - loss: 0.5484 - mean_absolute_error: 0.5991 - val_loss: 0.5102 - val_mean_absolute_error: 0.5726
Epoch 2/20
984/984 [==============================] - 11s 11ms/step - loss: 0.4813 - mean_absolute_error: 0.5702 - val_loss: 0.5245 - val_mean_absolute_error: 0.5943
Epoch 3/20
984/984 [==============================] - 11s 11ms/step - loss: 0.4772 - mean_absolute_error: 0.5681 - val_loss: 0.4984 - val_mean_absolute_error: 0.5755
Epoch 4/20
984/984 [==============================] - 11s 11ms/step - loss: 0.4766 - mean_absolute_error: 0.5673 - val_loss: 0.4988 - val_mean_absolute_error: 0.5702
Epoch 5/20
984/984 [==============================] - 11s 11ms/step - loss: 0.4704 - mean_absolute_error: 0.5635 - val_loss: 0.5112 - val_mean_absolute_error: 0.5849

training model 2
Epoch 1/20
983/983 [==============================] - 13s 12ms/step - loss: 0.5284 - mean_absolute_error: 0.5903 - val_loss: 0.5677 - v

In [24]:
dense_loss24,dense_test24 = dense_model.evaluate(test_windows24,test_labels24)
dense_loss72,dense_test72 = dense_model.evaluate(test_windows72,test_labels72)
dense_loss120,dense_test120 = dense_model.evaluate(test_windows120,test_labels120)

137/137 [==============================] - 1s 7ms/step - loss: 0.4759 - mean_absolute_error: 0.5484


## LSTM Model

In [25]:
lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(units=1)
])

In [26]:
compile_and_fit(lstm_model)

training model 1
Epoch 1/20
984/984 [==============================] - 15s 11ms/step - loss: 0.1161 - mean_absolute_error: 0.2280 - val_loss: 0.0569 - val_mean_absolute_error: 0.1472
Epoch 2/20
984/984 [==============================] - 10s 10ms/step - loss: 0.0336 - mean_absolute_error: 0.1266 - val_loss: 0.0455 - val_mean_absolute_error: 0.1277
Epoch 3/20
984/984 [==============================] - 11s 11ms/step - loss: 0.0311 - mean_absolute_error: 0.1198 - val_loss: 0.0433 - val_mean_absolute_error: 0.1287
Epoch 4/20
984/984 [==============================] - 11s 11ms/step - loss: 0.0297 - mean_absolute_error: 0.1157 - val_loss: 0.0375 - val_mean_absolute_error: 0.1132
Epoch 5/20
984/984 [==============================] - 10s 10ms/step - loss: 0.0297 - mean_absolute_error: 0.1145 - val_loss: 0.0369 - val_mean_absolute_error: 0.1142
Epoch 6/20
984/984 [==============================] - 11s 11ms/step - loss: 0.0280 - mean_absolute_error: 0.1103 - val_loss: 0.0342 - val_mean_absolute_e

(<tensorflow.python.keras.callbacks.History at 0x1b48b742c88>,
 <tensorflow.python.keras.callbacks.History at 0x1b6d2d9e988>)

In [27]:
lstm_loss24,lstm_test24 = lstm_model.evaluate(test_windows24,test_labels24)
lstm_loss72,lstm_test72 = lstm_model.evaluate(test_windows72,test_labels72)
lstm_loss120,lstm_test120 = lstm_model.evaluate(test_windows120,test_labels120)

137/137 [==============================] - 1s 7ms/step - loss: 0.0119 - mean_absolute_error: 0.0790


# Multi-Step Prediction

## Data Set Creation

In [28]:
# Function to create windows and labels
def create_dataset_multiple(train_df,window_size,label_size):
    train_windows = []
    train_labels = []
    
#     train_df = train_df.to_numpy()

    for i in range(len(train_df)-window_size-label_size):
        train_windows.append(train_df.iloc[i:i+window_size])
        train_labels.append(train_df['temperature'].iloc[i+window_size:i+label_size+window_size])
    
    return np.array(train_windows),np.array(train_labels)

In [29]:
#Train, val, test of size (24,24), (72,72), (120,120)
train_windows_multiple24, train_labels_multiple24 = create_dataset_multiple(train_df,24,24)
train_windows_multiple72, train_labels_multiple72 = create_dataset_multiple(train_df,72,72)
train_windows_multiple120, train_labels_multiple120 = create_dataset_multiple(train_df,120,120)


val_windows_multiple24, val_labels_multiple24 = create_dataset_multiple(val_df,24,24)
val_windows_multiple72, val_labels_multiple72 = create_dataset_multiple(val_df,72,72)
val_windows_multiple120, val_labels_multiple120 = create_dataset_multiple(val_df,120,120)

test_windows_multiple24, test_labels_multiple24 = create_dataset_multiple(test_df,24,24)
test_windows_multiple72, test_labels_multiple72 = create_dataset_multiple(test_df,72,72)
test_windows_multiple120, test_labels_multiple120 = create_dataset_multiple(test_df,120,120)

## Multi Baseline Model (Repeated)

In [30]:
def baseline_multi(data):
    return data[:,:,0]
    
baseline_multi24 = mean_absolute_error(baseline_multi(test_windows_multiple24),test_labels_multiple24)
baseline_multi72 = mean_absolute_error(baseline_multi(test_windows_multiple72),test_labels_multiple72)
baseline_multi120 = mean_absolute_error(baseline_multi(test_windows_multiple120),test_labels_multiple120)

baseline_multi24,baseline_multi72,baseline_multi120

(0.21222301797066634, 0.39038818488771215, 0.4387629334710285)

## Multi Linear Model

In [31]:
multi_linear_model24 = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    tf.keras.layers.Dense(24),
])

multi_linear_model72 = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    tf.keras.layers.Dense(72),
])

multi_linear_model120 = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    tf.keras.layers.Dense(120),
])

In [32]:
#Compile and Fit function for multi step models
def compile_and_fit_multi(model,tw,tl,vw,vl):
    model.compile(loss=tf.losses.MeanSquaredError(),metrics=[tf.metrics.MeanAbsoluteError()])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2, mode='min')
    model.fit(tw,tl,validation_data=(vw,vl), callbacks=[early_stopping],epochs=20)

In [33]:
compile_and_fit_multi(multi_linear_model24,
                      train_windows_multiple24,train_labels_multiple24,
                      val_windows_multiple24,val_labels_multiple24)
print()
compile_and_fit_multi(multi_linear_model72,
                      train_windows_multiple72,train_labels_multiple72,
                      val_windows_multiple72,val_labels_multiple72)
print()
compile_and_fit_multi(multi_linear_model120,
                      train_windows_multiple120,train_labels_multiple120,
                      val_windows_multiple120,val_labels_multiple120)

Epoch 1/20
984/984 [==============================] - 7s 7ms/step - loss: 1.1603 - mean_absolute_error: 0.8593 - val_loss: 0.9683 - val_mean_absolute_error: 0.7663
Epoch 2/20
984/984 [==============================] - 7s 7ms/step - loss: 1.0045 - mean_absolute_error: 0.8037 - val_loss: 0.9828 - val_mean_absolute_error: 0.7732
Epoch 3/20
984/984 [==============================] - 7s 7ms/step - loss: 1.0003 - mean_absolute_error: 0.8029 - val_loss: 0.9782 - val_mean_absolute_error: 0.7702

Epoch 1/20
981/981 [==============================] - 8s 7ms/step - loss: 1.0437 - mean_absolute_error: 0.8182 - val_loss: 0.9962 - val_mean_absolute_error: 0.7779
Epoch 2/20
981/981 [==============================] - 7s 7ms/step - loss: 0.9958 - mean_absolute_error: 0.8002 - val_loss: 0.9962 - val_mean_absolute_error: 0.7776
Epoch 3/20
981/981 [==============================] - 7s 7ms/step - loss: 0.9947 - mean_absolute_error: 0.7998 - val_loss: 0.9844 - val_mean_absolute_error: 0.7742
Epoch 4/20
981/

In [34]:
multi_linear_loss24,multi_linear_test24 = multi_linear_model24.evaluate(test_windows_multiple24,test_labels_multiple24)
multi_linear_loss72,multi_linear_test72 = multi_linear_model72.evaluate(test_windows_multiple72,test_labels_multiple72)
multi_linear_loss120,multi_linear_test120 = multi_linear_model120.evaluate(test_windows_multiple120,test_labels_multiple120)

134/134 [==============================] - 1s 4ms/step - loss: 1.1069 - mean_absolute_error: 0.8143


## Multi LSTM Model

In [35]:
multi_lstm_model24 = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(units = 24)
])

multi_lstm_model72 = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(units = 72)
])

multi_lstm_model120 = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(units = 120)
])


In [36]:
compile_and_fit_multi(multi_lstm_model24,
                      train_windows_multiple24,train_labels_multiple24,
                      val_windows_multiple24,val_labels_multiple24)
print()
compile_and_fit_multi(multi_lstm_model72,
                      train_windows_multiple72,train_labels_multiple72,
                      val_windows_multiple72,val_labels_multiple72)
print()
compile_and_fit_multi(multi_lstm_model120,
                      train_windows_multiple120,train_labels_multiple120,
                      val_windows_multiple120,val_labels_multiple120)

Epoch 1/20
984/984 [==============================] - 15s 12ms/step - loss: 0.3044 - mean_absolute_error: 0.3939 - val_loss: 0.1729 - val_mean_absolute_error: 0.2952
Epoch 2/20
984/984 [==============================] - 11s 11ms/step - loss: 0.1046 - mean_absolute_error: 0.2384 - val_loss: 0.1404 - val_mean_absolute_error: 0.2640
Epoch 3/20
984/984 [==============================] - 11s 11ms/step - loss: 0.0929 - mean_absolute_error: 0.2236 - val_loss: 0.1353 - val_mean_absolute_error: 0.2647
Epoch 4/20
984/984 [==============================] - 10s 10ms/step - loss: 0.0902 - mean_absolute_error: 0.2195 - val_loss: 0.1290 - val_mean_absolute_error: 0.2511
Epoch 5/20
984/984 [==============================] - 10s 10ms/step - loss: 0.0878 - mean_absolute_error: 0.2159 - val_loss: 0.1327 - val_mean_absolute_error: 0.2586
Epoch 6/20
984/984 [==============================] - 10s 10ms/step - loss: 0.0857 - mean_absolute_error: 0.2136 - val_loss: 0.1400 - val_mean_absolute_error: 0.2636

Epo

In [37]:
multi_lstm_loss24,multi_lstm_test24 = multi_lstm_model24.evaluate(test_windows_multiple24,test_labels_multiple24)
multi_lstm_loss72,multi_lstm_test72 = multi_lstm_model72.evaluate(test_windows_multiple72,test_labels_multiple72)
multi_lstm_loss120,multi_lstm_test120 = multi_lstm_model120.evaluate(test_windows_multiple120,test_labels_multiple120)

134/134 [==============================] - 1s 7ms/step - loss: 0.2275 - mean_absolute_error: 0.3529


# Results

In [38]:
single_index = ['Baseline(Last Step)','Linear','Dense','LSTM']
columns = ['Window Size 24','Window Size 72','Window Size 120']
scores24 = [baseline_test24,linear_test24,dense_test24,lstm_test24]
scores72 = [baseline_test72,linear_test72,dense_test72,lstm_test72]
scores120 = [baseline_test120,linear_test120,dense_test120,lstm_test120]

singlestep = pd.DataFrame([scores24,scores72,scores120],index=columns,columns=single_index).transpose()

heading_properties = [('font-size', '18px')]

cell_properties = [('font-size', '16px')]

dfstyle = [dict(selector="th", props=heading_properties),\
 dict(selector="td", props=cell_properties)]
singlestep.style.set_table_styles(dfstyle)

,Window Size 24,Window Size 72,Window Size 120
Baseline(Last Step),0.131695,0.131968,0.131428
Linear,0.518141,0.532831,0.541920
Dense,0.529515,0.541989,0.548427
LSTM,0.078646,0.079106,0.078998


In [39]:
multi_index = ['Baseline(Repeat)','Linear','LSTM']
columns_m = ['Window and Label Size 24','Window and Label Size 72','Window and Label Size 120']
scores_m24 = [baseline_multi24,multi_linear_test24,multi_lstm_test24]
scores_m72 = [baseline_multi72,multi_linear_test72,multi_lstm_test72]
scores_m120 = [baseline_multi120,multi_linear_test120,multi_lstm_test120]

multistep = pd.DataFrame([scores_m24,scores_m72,scores_m120],index=columns_m,columns=multi_index).transpose()

multistep.style.set_table_styles(dfstyle)

,Window and Label Size 24,Window and Label Size 72,Window and Label Size 120
Baseline(Repeat),0.212223,0.390388,0.438763
Linear,0.803180,0.808032,0.814345
LSTM,0.221473,0.292976,0.352899


In [89]:
kaggleset = pd.read_csv('ps6_test.csv')
#Joining wind direction and wind speed to wind vector
wd_rad = kaggleset.wind_direction*np.pi/180
kaggleset['Wx'] = kaggleset.wind_speed*np.cos(wd_rad)
kaggleset['Wy'] = kaggleset.wind_speed*np.sin(wd_rad)

kaggleset = kaggleset.drop(['wind_direction','wind_speed'],axis=1)

kaggleset['datetime'] = pd.to_datetime(kaggleset['datetime'])
timestamp_h1 = kaggleset['datetime'].map(datetime.datetime.timestamp)/3600

day = 24
year = (365.2425)*day

kaggleset['Day sin'] = np.sin(timestamp_h1 * (2 * np.pi / day))
kaggleset['Day cos'] = np.cos(timestamp_h1 * (2 * np.pi / day))
kaggleset['Year sin'] = np.sin(timestamp_h1 * (2 * np.pi / year))
kaggleset['Year cos'] = np.cos(timestamp_h1 * (2 * np.pi / year))

kaggleset = kaggleset.drop(['weather','datetime'],axis=1)

for i in range(120):
    kaggleset['temperature'].iloc[i] = (float(kaggleset['temperature'].iloc[i]) - train_mean['temperature']) / train_std['temperature']

for col in ['humidity','pressure','Wx','Wy','Day sin','Day cos','Year sin','Year cos']:
    kaggleset[col] = (kaggleset[col] - train_mean[col])/train_std[col]

kaggleset['temperature'] = kaggleset['temperature'].replace('?',-9999)
kaggleset

C:\Users\Divyam Khanna\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,temperature,humidity,pressure,Wx,Wy,Day sin,Day cos,Year sin,Year cos
0,0.861852,-1.602703,-0.480302,0.091631,-1.300145,0.999934,-0.999895,-0.949455,0.974841
1,0.436449,-1.420340,-0.480302,-0.276444,-1.276811,0.707021,-1.224618,-0.948693,0.975512
2,0.086845,-1.648294,-0.250485,0.091631,-0.532188,0.365917,-1.365884,-0.947931,0.976183
3,-0.009064,0.494472,-0.250485,0.091631,0.235769,-0.000133,-1.414068,-0.947168,0.976854
4,-0.173037,-0.782069,-0.250485,1.035947,0.584414,-0.366183,-1.365884,-0.946405,0.977524
...,...,...,...,...,...,...,...,...,...
235,-9999.000000,-1.693885,-0.250485,1.093720,0.485791,1.224694,0.707043,-0.758388,1.116554
236,-9999.000000,-1.830657,-0.250485,1.093720,0.485791,1.365984,0.365995,-0.757527,1.117085
237,-9999.000000,-1.739475,-0.365393,1.093720,0.485791,1.414175,0.000006,-0.756666,1.117615
238,-9999.000000,-2.149792,-0.250485,-1.900199,-0.289545,1.365984,-0.365983,-0.755804,1.118145


In [90]:
def pred(window):
    return lstm_model.predict(window.to_numpy().reshape(-1,120,9).astype('float64'))[0][0]

In [91]:
for i in range(120):
    value = pred(kaggleset[i:i+120])
#     print(kaggleset[i:i+120])
    kaggleset.at[i+120, 'temperature'] = value

In [94]:
ans = kaggleset['temperature'][120:]

In [104]:
k = pd.DataFrame(ans*train_std['temperature'] + train_mean['temperature']).reset_index()
k = k.drop('index',axis=1).reset_index()
k.columns=['Id','Predicted']

k

,Id,Predicted
0,0,299.188582
1,1,297.590463
2,2,296.193212
3,3,294.973288
4,4,293.989738
...,...,...
115,115,295.339291
116,116,296.421828
117,117,296.952904
118,118,296.937486


In [105]:
k.to_csv('lstm_single.csv',index=False)